# Data Collection

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('../data/hubway.db') # object to establish connection to SQLite db

In [2]:
cursor = conn.cursor()
print('Connection to database successful.')

Connection to database successful.


## Retrieve metadata

In [3]:
def list_tables(query):
    for row in cursor.execute(query):
        print(row)

In [4]:
query = 'SELECT name FROM sqlite_master WHERE type="table";'
print('\nDatabase tables:\n')
list_tables(query)


Database tables:

('trips',)
('stations',)


In [5]:
def fetchall_table(query):
    cursor.execute(query)
    attributes = [ x[0] for x in cursor.description]
    rows = cursor.fetchall()
    dataframe_variable = pd.DataFrame(rows, columns = attributes)
    return dataframe_variable

In [6]:
query = 'PRAGMA table_info("trips");'
print('\nTrips table info:')
fetchall_table(query)


Trips table info:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,duration,INTEGER,0,None,0
2,2,start_date,DATETIME,0,None,0
3,3,start_station,INTEGER,0,None,0
4,4,end_date,DATETIME,0,None,0
5,5,end_station,INTEGER,0,None,0
6,6,bike_number,TEXT,0,None,0
7,7,sub_type,TEXT,0,None,0
8,8,zip_code,TEXT,0,None,0
9,9,birth_date,REAL,0,None,0


In [7]:
query = 'PRAGMA table_info("stations");'
print('\nStations table info:')
fetchall_table(query)


Stations table info:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,station,TEXT,0,None,0
2,2,municipality,TEXT,0,None,0
3,3,lat,,0,None,0
4,4,lng,,0,None,0


In [8]:
query = 'PRAGMA foreign_key_list("trips");'
print('\nForeign keys in Trips table:')
fetchall_table(query)


Foreign keys in Trips table:


,id,seq,table,from,to,on_update,on_delete,match


The stations table's id column, which uniquely identifies each station, corresponds to the start_station and end-station columns in the trips table.  However, as the query shows, the tables are not actually linked.

## Data Analysis

1.  What was the duration of the longest trip?
2.  How many trips were taken by 'registered' users?
3.  What was the average trip duration?
4.  Do registered or casual users take longer trips?
5.  Which bike was used for the most trips?
6.  What is the average duration of trips by users over the age of 30?
7.  Which station is the most frequent starting point?
8.  Which stations are most frequently used for round trips?
9.  How many trips start and end in different municipalities?

In [9]:
def run_query(query):
    return pd.read_sql_query(query, conn) # takes query stored as a string variable as input to return formatted dataframe

In [10]:
query = '''SELECT duration FROM trips
            ORDER BY duration DESC
            LIMIT 5;'''
run_query(query)

,duration
0,9999
1,9998
2,9998
3,9997
4,9996


The duration of the longest trip is 9999 seconds, which is the only record of this duration indicating that it is likely not a maximum value  (4-digit number) defined by the table.  It is plausible that the company decided 4 digits would be sufficient to track majority of trips given the additional fees charged for rides over 30 minutes, or 1800 seconds.

In [11]:
query = '''SELECT sub_type, COUNT(*) AS "Total Number of Trips taken by Users"
            FROM trips
            GROUP BY sub_type;'''
run_query(query)

,sub_type,Total Number of Trips taken by Users
0,Casual,464809
1,Registered,1105192


There have been 1105192 trips taken by 'registered' users, and 464809 trips taken by 'casual' users, which make up around 42% of Hubway's users.  Hubway could therefore target this segment of casual users to better inform them of their registered user prices to help them avoid overage charges.

In [12]:
query = '''SELECT AVG(duration) AS "Average Duration of Trips taken by Users"
            FROM trips;'''
run_query(query)

,Average Duration of Trips taken by Users
0,912.409682


The average trip duration was about 912 seconds, or around 15 minutes, which seems right given the extra charges for trips over 30 minutes.  Hubway's service is designed for short, one-way trips.

In [13]:
query = '''SELECT sub_type, AVG(duration) AS "Average Duration of Trips taken by Users"
            FROM trips
            GROUP BY sub_type;'''
run_query(query)

,sub_type,Average Duration of Trips taken by Users
0,Casual,1519.643897
1,Registered,657.026067


On average, casual users take trips that last nearly 25 minutes per ride, whereas registered users take trips that last around 11 minutes per ride.  There is a probability that registered users are taking shorter trips more frequently, whereas casual users take longer trips less frequently by comparison.  The difference in the data may indicate different user demographics, eg. locals vs. tourists, which Hubway may investigate further.

In [14]:
query = '''SELECT bike_number AS "Bike Number", COUNT(*) AS "Number of Trips"
            FROM trips
            GROUP BY bike_number
            ORDER BY COUNT(*) DESC
            LIMIT 1;'''
run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


Bike number B00490 has taken the most trips.  Hubway could use this reference among others to assess whether the bike needs maintenance, and to help appraise the condition of the rest of their fleet.

In [15]:
query = '''SELECT AVG(duration) AS "Average Duration"
            FROM trips
            WHERE (2021 - birth_date) > 30;'''
run_query(query)

,Average Duration
0,915.167566


The average duration of users over the age of 30 is around 15 minutes.  Analyzing this central tendency across different age groups could help Hubway find insights about different age demographics.  The same type of user analysis can be done with attributes such as gender, and zip code to help Hubway better understand its customers.

In [16]:
query = '''SELECT stations.station AS "Name of Station", COUNT(*) AS "Frequency"
            FROM trips INNER JOIN stations
            ON trips.start_station = stations.id
            GROUP BY stations.station
            ORDER BY COUNT(*) DESC
            LIMIT 5;'''
run_query(query)

,Name of Station,Frequency
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


The South Station on 700 Atlantic Ave. is the most frequent starting point for users.  Hubway could use this information to assess whether the station needs maintenance, or possibly expansion to better accommodate users at this station and others frequently used as starting points.  The stations' location can be examined for similarities to determine opportunities for other station locations.

In [17]:
query = '''SELECT stations.station AS "Name of Station", COUNT(*) AS "Frequency"
            FROM trips INNER JOIN stations
            ON trips.start_station = stations.id
            WHERE trips.start_station = trips.end_station
            GROUP BY stations.station
            ORDER BY COUNT(*) DESC
            LIMIT 5;'''
run_query(query)

,Name of Station,Frequency
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


As the query shows, the majority of the stations used most frequently for round-trips are among the most busiest (as shown from the previous query).  However, the significantly lower frequencies suggest that users typically trips that take them from point A to B, as opposed to cycling around before returning to the same station they started from.  The Esplanade on Beacon/Arlington St. is the most popular station for round-trips, but is not amont the most busiest.  As the name of the station suggests, it is probably located in an area ideal for leisurely bike rides.  Once again, the location can be evaluated to determine opportunities for other station locations.

In [18]:
query = '''SELECT COUNT(trips.id) AS "Frequency of Trips between different Municipalities"
            FROM trips INNER JOIN stations AS "Start"
            ON trips.start_station = Start.id
            INNER JOIN stations AS "End"
            ON trips.end_station = End.id
            WHERE Start.municipality != End.municipality;'''
run_query(query)

,Frequency of Trips between different Municipalities
0,309748


In [19]:
query = '''SELECT count(*) AS "Total Number of Trips"
            FROM trips;'''
run_query(query)

,Total Number of Trips
0,1570001


There are about 300000 out of around 1.5 million trips, or 20%, that start and end in different municipalities indicating that Hubway's services are used more regularly for short trips (within a single municipality) than long trips between municipalities.

In [20]:
cursor.close()
print('Connection to database successfully closed.')

Connection to database successfully closed.
